### Get the data from Wiki page and create the first Dataframe with Postal Codes and neighbourhoods

In [1]:
import requests
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

In [3]:
results = soup.find('table', attrs={'class':'wikitable sortable'})

In [4]:
results = results.find('tbody').find_all('tr')

In [5]:
print(results[0].contents)
print(results[0].contents[1].text, results[0].contents[3].text,results[0].contents[5].text)

['\n', <th>Postcode</th>, '\n', <th>Borough</th>, '\n', <th>Neighbourhood
</th>]
Postcode Borough Neighbourhood



In [6]:
records = []
lastpost = ''
lastneighbourhood= ''
for result in results:
    postcode = result.contents[1].text
    borough = result.contents[3].text
    neighbourhood = result.contents[5].text[0:-1]
    if borough != 'Not assigned' and postcode != 'Postcode':
        records.append((postcode, borough, neighbourhood))

In [7]:
import pandas as pd
df = pd.DataFrame(records, columns=['Postcode', 'Borough', 'Neighbourhood'])    

df = df.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ', '.join(x))
df = df.reset_index()
print(df.shape)

(103, 3)


In [8]:
df.sort_values(by='Postcode', inplace = True)
print(df.dtypes)
print(df.head())

Postcode         object
Borough          object
Neighbourhood    object
dtype: object
   Postcode      Borough                           Neighbourhood
6       M1B  Scarborough                          Rouge, Malvern
12      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
18      M1E  Scarborough       Guildwood, Morningside, West Hill
22      M1G  Scarborough                                  Woburn
26      M1H  Scarborough                               Cedarbrae


#### First Dataframe with PostalCodes is ready. 
### Now Let us get the Lang and Lat and join these

In [9]:
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
 # g = geocoder.google('{}, Toronto, Ontario'.format('M4E'))
  #lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

## GEOCODER Didn't work here. to using the refence CSV file for GeoCodes

In [11]:
geodata = pd.read_csv("http://cocl.us/Geospatial_data")
geodata.head(20)
#print(geodata['Postal Code'].dtype)
# geodata.dtypes

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [12]:
geodata.sort_values(['Postal Code'])
geodata.head()
# Very Important to sort both the data frames before joining.
#print(geodata.iloc[0, 0], type(geodata.iloc[0, 0]))

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
dfll = pd.merge(df, geodata, left_on='Postcode', right_on='Postal Code', how='left')
dfll.drop(['Postal Code'], axis=1, inplace = True)
dfll.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
dfll.shape

(103, 5)

#### Here we are - Our final dataframe is ready with Postal details and corresponding Long and Lat.
### Time to start exploring the neighborhood